In [1]:
import math
import random
class BigBase65536:
    BASE = 65536  # 2^16

    def __init__(self, digits=None, negative=False):
        if digits is None:
            digits = []
        self.digits = digits  # list of 0..65535
        self.negative = negative
        self._trim_leading_zeros()

    def _trim_leading_zeros(self):
        while len(self.digits) > 1 and self.digits[-1] == 0:
            self.digits.pop()
        if len(self.digits) == 1 and self.digits[0] == 0:
            self.negative = False


    @staticmethod
    def divmod_small(a: "BigBase65536", small_val: int) -> tuple["BigBase65536", int]:
        """
        Chunk-based divmod by small_val (<= 65535). Returns (quotient, remainder).
        quotient is BigBase65536, remainder is a standard int in [0..small_val-1].
        """
        remainder = 0
        out_digits = [0]*len(a.digits)

        for i in reversed(range(len(a.digits))):
            # combine remainder with current digit
            current = (remainder << 16) + a.digits[i]
            qdigit = current // small_val
            remainder = current % small_val
            out_digits[i] = qdigit

        quotient = BigBase65536(out_digits, negative=a.negative)
        quotient._trim_leading_zeros()

        return (quotient, remainder)

    def to_decimal_str(self) -> str:
        """
        Convert this BigBase65536 to a decimal string,
        purely via repeated divmod by 10. No Python int(...).
        """
        # Check if zero:
        if len(self.digits) == 1 and self.digits[0] == 0:
            return "0"

        # Make a copy so we don't mutate self
        tmp_digits = self.digits[:]
        tmp = BigBase65536(tmp_digits, negative=False)
        neg = self.negative

        digits_rev = []
        while not (len(tmp.digits) == 1 and tmp.digits[0] == 0):
            tmp, rem = self.divmod_small(tmp, 10)
            digits_rev.append(str(rem))

        decimal_str = "".join(reversed(digits_rev))
        if neg:
            decimal_str = "-" + decimal_str
        return decimal_str

    @classmethod
    def from_int(cls, value: int) -> "BigBase65536":
        if value == 0:
            return cls([0], negative=False)
        negative = (value < 0)
        value = abs(value)
        digits = []
        while value > 0:
            digits.append(value % cls.BASE)
            value //= cls.BASE
        return cls(digits, negative=negative)

    def to_int(self) -> int:
        result = 0
        mul = 1
        for chunk in self.digits:
            result += chunk * mul
            mul *= self.BASE
        return -result if self.negative else result

    def __repr__(self):
        # Let to_decimal_str() handle the sign and preserve large digits
        return f"<BigBase65536 {self.to_decimal_str()}>"

    @staticmethod
    def _compare_abs(a: "BigBase65536", b: "BigBase65536"):
        """
        Compare absolute values of a and b.
        Return +1 if |a| > |b|, -1 if |a| < |b|, 0 if equal.
        """
        if len(a.digits) > len(b.digits):
            return 1
        elif len(a.digits) < len(b.digits):
            return -1
        else:
            for i in reversed(range(len(a.digits))):
                if a.digits[i] > b.digits[i]:
                    return 1
                elif a.digits[i] < b.digits[i]:
                    return -1
            return 0

    @staticmethod
    def _subtract_abs(big, small):
        """
        Subtract |small| from |big|, assuming |big| >= |small|.
        Sign of result = big.negative
        """
        result_digits = []
        carry = 0
        for i in range(len(big.digits)):
            da = big.digits[i]
            db = small.digits[i] if i < len(small.digits) else 0
            s = da - db - carry
            if s < 0:
                s += 65536
                carry = 1
            else:
                carry = 0
            result_digits.append(s & 0xFFFF)

        res = BigBase65536(result_digits, negative=big.negative)
        res._trim_leading_zeros()
        return res

    @staticmethod
    def add(a: "BigBase65536", b: "BigBase65536") -> "BigBase65536":
        if a.negative == b.negative:
            # Same sign => add magnitudes
            result_digits = []
            carry = 0
            max_len = max(len(a.digits), len(b.digits))
            for i in range(max_len):
                da = a.digits[i] if i < len(a.digits) else 0
                db = b.digits[i] if i < len(b.digits) else 0
                s = da + db + carry
                carry = s >> 16
                s &= 0xFFFF
                result_digits.append(s)
            if carry:
                result_digits.append(carry)
            return BigBase65536(result_digits, negative=a.negative)
        else:
            # Opposite signs => subtract smaller from bigger
            cmp_val = BigBase65536._compare_abs(a, b)
            if cmp_val == 0:
                return BigBase65536([0], negative=False)
            elif cmp_val > 0:
                # |a| > |b| => result sign = a.negative
                return BigBase65536._subtract_abs(a, b)
            else:
                # |b| > |a| => result sign = b.negative
                return BigBase65536._subtract_abs(b, a)

    @staticmethod
    def mul(a: "BigBase65536", b: "BigBase65536") -> "BigBase65536":
        negative = (a.negative != b.negative)
        result_len = len(a.digits) + len(b.digits)
        result_digits = [0]*result_len

        for i, da in enumerate(a.digits):
            carry = 0
            for j, db in enumerate(b.digits):
                mul_val = da * db + result_digits[i+j] + carry
                result_digits[i+j] = mul_val & 0xFFFF
                carry = mul_val >> 16
            if carry:
                result_digits[i + len(b.digits)] += carry

        res = BigBase65536(result_digits, negative=negative)
        res._trim_leading_zeros()
        return res

In [2]:
def gcd_int(a: int, b: int) -> int:
    """Standard Euclidean algorithm for Python int."""
    while b != 0:
        a, b = b, a % b
    return abs(a)

class MyFraction:
    """
    Minimal fraction class that uses BigBase65536
    for numerator/denominator but uses an int-based gcd for simplification.
    """
    def __init__(self, numerator: "BigBase65536", denominator: "BigBase65536"):
        self.numerator = numerator
        self.denominator = denominator    
        # If None => default 0
        if numerator is None:
            numerator = BigBase65536.from_int(0)
        if denominator is None:
            # default to 1
            denominator = BigBase65536.from_int(1)

        # Zero denominator => error
        if denominator.to_int() == 0:
            raise ZeroDivisionError("MyFraction denominator cannot be zero.")

        self.numerator = numerator   # BigBase65536
        self.denominator = denominator  # BigBase65536

        self._normalize()

    def _normalize(self):
        """
        1) If denominator < 0, flip signs so denominator is positive.
        2) gcd-reduce if possible (requires gcd on Python int,
           or a custom gcd on BigBase65536).
        """
        # 1) Ensure denominator is always >= 0 (like Fraction does)
        denom_int = self.denominator.to_int()
        if denom_int < 0:
            # flip sign
            self.denominator.negative = False  # make denominator positive
            # flip numerator sign
            self.numerator.negative = not self.numerator.negative

        # 2) Try to reduce by gcd if they're not too large
        #    (For a fully custom approach, you'd write a gcd for BigBase65536).
        num_int = self.numerator.to_int()
        den_int = self.denominator.to_int()

        # gcd with Python int (works if within Python int range):
        g = gcd_int(num_int, den_int)
        if g > 1:
            # divide numerator, denominator by gcd
            new_num = num_int // g
            new_den = den_int // g
            self.numerator = BigBase65536.from_int(new_num)
            self.denominator = BigBase65536.from_int(new_den)

    @classmethod
    def from_int(cls, val: int) -> "MyFraction":
        return cls(BigBase65536.from_int(val), BigBase65536.from_int(1))

    def to_int(self) -> int:
        """
        Convert fraction to an integer (flooring is *not* done, 
        we just do integer division if denominator==1).
        """
        den = self.denominator.to_int()
        if den != 1:
            # Not an integer fraction => do floor
            return self.numerator.to_int() // den
        else:
            return self.numerator.to_int()

    def __add__(self, other: "MyFraction") -> "MyFraction":
        """
        (a/b) + (c/d) = (ad + bc) / bd
        """
        a = self.numerator
        b = self.denominator
        c = other.numerator
        d = other.denominator

        ad = BigBase65536.mul(a, d)
        bc = BigBase65536.mul(b, c)
        new_num = BigBase65536.add(ad, bc)
        new_den = BigBase65536.mul(b, d)
        return MyFraction(new_num, new_den)

    def __sub__(self, other: "MyFraction") -> "MyFraction":
        """
        (a/b) - (c/d) = (ad - bc) / (bd)
        """
        a = self.numerator
        b = self.denominator
        c = other.numerator
        d = other.denominator

        ad = BigBase65536.mul(a, d)
        bc = BigBase65536.mul(b, c)

        # Effectively do (ad) - (bc) by negating bc and adding:
        neg_bc = BigBase65536(bc.digits[:], negative=(not bc.negative))
        new_num = BigBase65536.add(ad, neg_bc)

        new_den = BigBase65536.mul(b, d)
        return MyFraction(new_num, new_den)

    def __mul__(self, other: "MyFraction") -> "MyFraction":
        # (a/b) * (c/d) = ac / bd
        new_num = BigBase65536.mul(self.numerator, other.numerator)
        new_den = BigBase65536.mul(self.denominator, other.denominator)
        return MyFraction(new_num, new_den)

    def __truediv__(self, other: "MyFraction") -> "MyFraction":
        # (a/b) / (c/d) = (a/b) * (d/c) = ad / bc
        if other.numerator.to_int() == 0:
            raise ZeroDivisionError("Cannot divide by zero fraction.")
        new_num = BigBase65536.mul(self.numerator, other.denominator)
        new_den = BigBase65536.mul(self.denominator, other.numerator)
        return MyFraction(new_num, new_den)

    def __eq__(self, other: "MyFraction") -> bool:
        return (self.numerator.to_int()   == other.numerator.to_int() and
                self.denominator.to_int() == other.denominator.to_int())

    def __str__(self):
        # e.g. "-123/456", or "789" if denominator=1
        den = self.denominator.to_int()
        num = self.numerator.to_int()
        if den == 1:
            return str(num)
        else:
            return f"{num}/{den}"

    def to_float(self) -> float:
        return self.numerator.to_int() / self.denominator.to_int()

In [3]:
class SqrtD:
    """
    Represents a + b*sqrt(D), 
    where a,b are MyFraction, 
    and D is an integer.
    """
    def __init__(self, a=None, b=None, D=0):
        if a is None: a = MyFraction.from_int(0)
        if b is None: b = MyFraction.from_int(0)
        self.a = a  # MyFraction
        self.b = b  # MyFraction
        self.D = D  # int (not BigBase65536 for simplicity)

    def __add__(self, other: "SqrtD") -> "SqrtD":
        # Must have same D
        if self.D != other.D:
            raise ValueError("Cannot add sqrt(D1) with sqrt(D2) for different D.")
        new_a = self.a + other.a
        new_b = self.b + other.b
        return SqrtD(new_a, new_b, self.D)

    def __sub__(self, other: "SqrtD") -> "SqrtD":
        if self.D != other.D:
            raise ValueError("Cannot subtract sqrt(D1) from sqrt(D2) for different D.")
        new_a = self.a - other.a
        new_b = self.b - other.b
        return SqrtD(new_a, new_b, self.D)

    def __mul__(self, other: "SqrtD") -> "SqrtD":
        if self.D != other.D:
            raise ValueError("Cannot multiply sqrt(D1) by sqrt(D2) for different D.")
        # (a + b sqrt(D)) * (c + d sqrt(D)) = (ac + bd*D) + (ad + bc)*sqrt(D)
        ac = self.a * other.a
        bdD = (self.b * other.b) * MyFraction.from_int(self.D)
        new_a = ac + bdD
        ad = self.a * other.b
        bc = self.b * other.a
        new_b = ad + bc
        return SqrtD(new_a, new_b, self.D)

    def __truediv__(self, other: "SqrtD") -> "SqrtD":
        if self.D != other.D:
            raise ValueError("Cannot divide sqrt(D1) by sqrt(D2) for different D.")
        # Rationalize: 
        # (a + b sqrt(D)) / (c + d sqrt(D)) 
        # = [(a + b sqrt(D)) * (c - d sqrt(D))] / [(c + d sqrt(D)) * (c - d sqrt(D))]
        # Denominator => c^2 - d^2*D
        denom = (other.a * other.a) - (other.b * other.b) * MyFraction.from_int(self.D)
        if denom.to_float() == 0.0:
            raise ZeroDivisionError("Division by zero in SqrtD.")
        # Numerator => (a + b sqrt(D))*(c - d sqrt(D))
        c_negd = SqrtD(other.a, MyFraction.from_int(-1)*other.b, self.D)
        numerator = self * c_negd
        new_a = numerator.a / denom
        new_b = numerator.b / denom
        return SqrtD(new_a, new_b, self.D)

    def to_float(self) -> float:
        """
        approximate: float(a) + float(b)*sqrt(D)
        """
        return self.a.to_float() + self.b.to_float()*math.sqrt(self.D)

    def __str__(self):
        return f"({self.a} + {self.b}*sqrt({self.D}))"

In [4]:
if __name__ == "__main__":
    # BigBase65536 negative test
    valA = BigBase65536.from_int(-123456)
    valB = BigBase65536.from_int(123)
    print("A =>", valA)  # -123456
    print("B =>", valB)  # 123
    sumAB = BigBase65536.add(valA, valB)
    print("A + B =>", sumAB, "(should be -123333)")

    # MyFraction usage
    frac1 = MyFraction.from_int(-1) / MyFraction.from_int(2)  # -1/2
    frac2 = MyFraction.from_int(3) / MyFraction.from_int(4)   # 3/4
    print("frac1 =>", frac1)  # e.g. "-1/2"
    print("frac2 =>", frac2)  # "3/4"
    print("frac1+frac2 =>", frac1+frac2)  # 1/4
    print("frac1*frac2 =>", frac1*frac2)  # -3/8

    # Symbolic SqrtD usage
    # x = -2 + 3 sqrt(5)
    x = SqrtD(MyFraction.from_int(-2), MyFraction.from_int(3), 5)
    print("x =>", x)
    print("x as float =>", x.to_float())  # ~ 4.7082

A => <BigBase65536 -123456>
B => <BigBase65536 123>
A + B => <BigBase65536 -123333> (should be -123333)
frac1 => -1/2
frac2 => 3/4
frac1+frac2 => 1/4
frac1*frac2 => -3/8
x => (-2 + 3*sqrt(5))
x as float => 4.708203932499369


In [5]:
def test_bigbase65536_basic():
    print("\n=== test_bigbase65536_basic ===")
    # Positive
    valA = BigBase65536.from_int(123456)
    valB = BigBase65536.from_int(654321)
    sumAB = BigBase65536.add(valA, valB)
    prodAB = BigBase65536.mul(valA, valB)
    print("A =>", valA, "\nB =>", valB)
    print("A + B =>", sumAB)
    print("A * B =>", prodAB)
    print("Check =>", sumAB.to_int(), prodAB.to_int())

    # Negative
    valNegA = BigBase65536.from_int(-123456)
    valNegB = BigBase65536.from_int(-654321)
    sumNegAB = BigBase65536.add(valNegA, valNegB)
    prodNegAB = BigBase65536.mul(valNegA, valNegB)
    print("\nNegA =>", valNegA, "\nNegB =>", valNegB)
    print("NegA + NegB =>", sumNegAB)
    print("NegA * NegB =>", prodNegAB)
    print("Check =>", sumNegAB.to_int(), prodNegAB.to_int())

    # Mixed signs
    sum_mix = BigBase65536.add(valNegA, valB)
    print("\nvalNegA + valB =>", sum_mix, "(Check =>", sum_mix.to_int(), ")")

def test_myfraction_basic():
    print("\n=== test_myfraction_basic ===")
    half_neg = MyFraction.from_int(-1) / MyFraction.from_int(2)         # -1/2
    three_quarters = MyFraction.from_int(3) / MyFraction.from_int(4)    # 3/4

    print("half_neg =>", half_neg)
    print("three_quarters =>", three_quarters)

    sum_f = half_neg + three_quarters
    print("\n(-1/2) + (3/4) =>", sum_f, " (~", sum_f.to_float(), ")")

    prod_f = half_neg * three_quarters
    print("(-1/2) * (3/4) =>", prod_f, " (~", prod_f.to_float(), ")")

    # Another fraction: 7/3
    fracA = MyFraction.from_int(7) / MyFraction.from_int(3)
    print("\n7/3 =>", fracA, "(~", fracA.to_float(), ")")

    # Division test => (7/3) / (3/4) => 28/9
    div_res = fracA / three_quarters
    print("(7/3) / (3/4) =>", div_res, "(~", div_res.to_float(), ")")

def test_sqrtd_basic():
    print("\n=== test_sqrtd_basic ===")
    # x = -2 + 3*sqrt(5)
    x = SqrtD(
        a=MyFraction.from_int(-2),
        b=MyFraction.from_int(3),
        D=5
    )
    print("x =>", x)
    print("x as float =>", x.to_float())

    # y = 4/5 + (-1/3)*sqrt(5)
    # build 4/5
    y_a = MyFraction(BigBase65536.from_int(4), BigBase65536.from_int(5))
    # build -1/3
    y_b = MyFraction(BigBase65536.from_int(-1), BigBase65536.from_int(3))
    y = SqrtD(a=y_a, b=y_b, D=5)
    print("\ny =>", y)
    print("y as float =>", y.to_float())

    # x + y
    sum_xy = x + y
    print("\nx + y =>", sum_xy, " (~", sum_xy.to_float(), ")")

    # x * y
    prod_xy = x * y
    print("x * y =>", prod_xy, " (~", prod_xy.to_float(), ")")

    # x / y
    try:
        div_xy = x / y
        print("x / y =>", div_xy, " (~", div_xy.to_float(), ")")
    except ZeroDivisionError:
        print("x / y => ZeroDivisionError")

In [6]:
import math

class Wave:
    """
    A wave with amplitude (rational or SqrtD) and phase (rational or SqrtD),
    specifying phase in radians. For complex representation, we do
    amplitude*(cos(phase) + i*sin(phase)).

    Because we're avoiding built-in Fraction, we assume your
    sin_maclaurin, cos_maclaurin are re-implemented in a custom module.
    """

    def __init__(self, amplitude=None, phase=None):
        # default => amplitude=0, phase=0
        if amplitude is None:
            # e.g. MyFraction 0
            amplitude = MyFraction.from_int(0)
        if phase is None:
            phase = MyFraction.from_int(0)
        self.amplitude = amplitude
        self.phase = phase  # must be rational or SqrtD with a rational "a" ?

    def multiply(self, other: "Wave") -> "Wave":
        """Multiply => amplitude multiply, phase add."""
        # amplitude = A1*A2
        new_amp = self.amplitude * other.amplitude
        # phase = p1 + p2
        new_phase = self.phase + other.phase
        return Wave(new_amp, new_phase)

    def add_wave(self, other: "Wave",
                 max_terms=40,
                 epsilon=MyFraction.from_int(1)): 
        """
        Add waves => interpret each wave as complex => sum => convert back.

        (You can store Maclaurin expansions for sin/cos in a custom method
         if your code doesn't rely on Python's math.sin/cos.)
        """
        real1, imag1 = self.to_complex(max_terms, epsilon)
        real2, imag2 = other.to_complex(max_terms, epsilon)
        sum_real = real1 + real2
        sum_imag = imag1 + imag2

        # Convert sum back to amplitude/phase => amplitude = sqrt(re^2+im^2),
        # phase=atan2(im, re). But you'll need a custom sqrt() on MyFraction or
        # fallback to float for angle, etc.

        # For demonstration, we'll do approximate float:
        re_f = float(sum_real.to_float())
        im_f = float(sum_imag.to_float())
        amp = math.hypot(re_f, im_f)
        ph = math.atan2(im_f, re_f)

        # Then store them back as MyFraction from float or a SqrtD approach:
        amp_fraction = MyFraction.from_int(int(amp))  # naive approach
        # Real approach would approximate better, or store in SqrtD

        ph_fraction = MyFraction.from_int(0)
        # Typically you'd do rational approximation of 'ph'.

        return Wave(amp_fraction, ph_fraction)



    def to_complex(self, max_terms=40, epsilon=MyFraction.from_int(1)):
        """
        amplitude*(cos(phase) + i*sin(phase)).
        If phase is a MyFraction, call your custom sin_maclaurin/cos_maclaurin.
        If phase is a SqrtD, you'd need a fancier approach.
        """
        # We'll do a simplified version:
        phase_float = self.phase.to_float()  # approximate
        sin_val = math.sin(phase_float)  # or custom Maclaurin if strictly no math
        cos_val = math.cos(phase_float)

        amp_float = self.amplitude.to_float()
        real = MyFraction.from_int(int(amp_float*cos_val))  # naive
        imag = MyFraction.from_int(int(amp_float*sin_val))

        return (real, imag)

    def __str__(self):
        return f"Wave(amp={self.amplitude}, phase={self.phase})"

In [7]:
import time
import random

def timing_tests(num_trials=20, max_terms=40):
    """
    Perform some basic timing on fraction-based sin expansions and wave arithmetic.
    Adjust num_trials, max_terms, or random ranges as needed.
    """

    print("\n=== timing_tests ===")
    # 1) Time random sin expansions with fraction-based approach
    start_sin = time.time()
    for _ in range(num_trials):
        angle_float = random.uniform(0, 2.0)
        # Convert to your fraction type
        angle_frac = MyFraction.from_int(int(angle_float*1000)) / MyFraction.from_int(1000)
        # call your Maclaurin sin => e.g. sin_maclaurin(angle_frac, max_terms=max_terms)
        # *If* your function is something like: sin_maclaurin(MyFraction) -> MyFraction
        # or you have to wrap it in SqrtD or something. This depends on your implementation.
        # We'll just do a placeholder call:
        # _ = sin_maclaurin(angle_frac, max_terms)  # for example
    end_sin = time.time()
    total_sin = end_sin - start_sin
    print(f"Fraction-based sin() expansions for {num_trials} random angles => "
          f"{total_sin:.4f} s, avg {total_sin/num_trials:.4f} s each")

    # 2) Time random wave multiplications
    start_wave_mult = time.time()
    for _ in range(num_trials):
        ampA = random.uniform(0.1, 3.0)
        phaseA = random.uniform(0, 2.0)
        ampB = random.uniform(0.1, 3.0)
        phaseB = random.uniform(0, 2.0)

        waveA = Wave(
            MyFraction.from_int(int(ampA*1000)) / MyFraction.from_int(1000),
            MyFraction.from_int(int(phaseA*1000)) / MyFraction.from_int(1000)
        )
        waveB = Wave(
            MyFraction.from_int(int(ampB*1000)) / MyFraction.from_int(1000),
            MyFraction.from_int(int(phaseB*1000)) / MyFraction.from_int(1000)
        )
        _ = waveA.multiply(waveB)
    end_wave_mult = time.time()
    total_wave_mult = end_wave_mult - start_wave_mult
    print(f"Wave multiplication for {num_trials} random pairs => "
          f"{total_wave_mult:.4f} s, avg {total_wave_mult/num_trials:.4f} s each")

    # 3) Time wave addition => triggers sin/cos expansions in to_complex
    start_wave_add = time.time()
    for _ in range(num_trials):
        ampA = random.uniform(0.1, 2.0)
        phaseA = random.uniform(0, 2.0)
        ampB = random.uniform(0.1, 2.0)
        phaseB = random.uniform(0, 2.0)
        waveA = Wave(
            MyFraction.from_int(int(ampA*1000)) / MyFraction.from_int(1000),
            MyFraction.from_int(int(phaseA*1000)) / MyFraction.from_int(1000)
        )
        waveB = Wave(
            MyFraction.from_int(int(ampB*1000)) / MyFraction.from_int(1000),
            MyFraction.from_int(int(phaseB*1000)) / MyFraction.from_int(1000)
        )
        _ = waveA.add_wave(waveB, max_terms=max_terms)
    end_wave_add = time.time()
    total_wave_add = end_wave_add - start_wave_add
    print(f"Wave addition for {num_trials} random pairs => "
          f"{total_wave_add:.4f} s, avg {total_wave_add/num_trials:.4f} s each")


def stress_test(num_trials=10, angle_range=(0, 100.0), max_terms=100):
    """
    A 'worst case' style stress test for fraction-based expansions:
      - random angles up to angle_range's max (could be large)
      - no explicit angle reduction => large expansions
    We'll measure:
      1) sin expansions timing
      2) wave addition timing
    """

    print("\n=== stress_test ===")
    print(f" - {num_trials} random angles/pairs")
    print(f" - angle range={angle_range}")
    print(f" - max_terms={max_terms}\n")

    # 1) Time random sin expansions
    start_sin = time.time()
    for _ in range(num_trials):
        angle_f = random.uniform(*angle_range)
        angle_frac = MyFraction.from_int(int(angle_f*1000)) / MyFraction.from_int(1000)
        # _ = sin_maclaurin(angle_frac, max_terms)   # placeholder
    end_sin = time.time()
    total_sin = end_sin - start_sin
    print(f"SIN expansions for {num_trials} large angles => "
          f"{total_sin:.4f} s total, avg {total_sin/num_trials:.4f} s each")

    # 2) Time wave additions => triggers expansions in to_complex
    start_wave_add = time.time()
    for _ in range(num_trials):
        ampA_f = random.uniform(1.0, 3.0)
        angleA_f = random.uniform(*angle_range)
        ampB_f = random.uniform(1.0, 3.0)
        angleB_f = random.uniform(*angle_range)

        waveA = Wave(
            MyFraction.from_int(int(ampA_f*1000)) / MyFraction.from_int(1000),
            MyFraction.from_int(int(angleA_f*1000)) / MyFraction.from_int(1000)
        )
        waveB = Wave(
            MyFraction.from_int(int(ampB_f*1000)) / MyFraction.from_int(1000),
            MyFraction.from_int(int(angleB_f*1000)) / MyFraction.from_int(1000)
        )
        _ = waveA.add_wave(waveB, max_terms=max_terms)
    end_wave_add = time.time()
    total_wave_add = end_wave_add - start_wave_add
    print(f"WAVE addition for {num_trials} large-angle pairs => "
          f"{total_wave_add:.4f} s total, avg {total_wave_add/num_trials:.4f} s each")

In [8]:
def test_wave_basic():
    print("\n=== test_wave_basic ===")

    # wave1 => amplitude= 3/2, phase= 1/2
    wave1_amp = MyFraction.from_int(3) / MyFraction.from_int(2)
    wave1_phase = MyFraction.from_int(1) / MyFraction.from_int(2)
    wave1 = Wave(wave1_amp, wave1_phase)
    print("wave1 =>", wave1)

    # wave2 => amplitude= -4/3, phase= 2/3
    wave2_amp = (MyFraction.from_int(-4) / MyFraction.from_int(3))
    wave2_phase = (MyFraction.from_int(2) / MyFraction.from_int(3))
    wave2 = Wave(wave2_amp, wave2_phase)
    print("wave2 =>", wave2)

    # multiply => amplitude multiply, phase add
    w_mult = wave1.multiply(wave2)
    print("\nwave1 * wave2 =>", w_mult)

    # add_wave => complex sum => amplitude+phase
    w_add = wave1.add_wave(wave2, max_terms=20)
    print("wave1 + wave2 =>", w_add)

    # wave1 => real/imag
    real1, imag1 = wave1.to_complex(max_terms=20)
    print("\nwave1 to_complex => real =", real1, ", imag =", imag1)
    print("   as float => real=%.5f, imag=%.5f" % (real1.to_float(), imag1.to_float()))

    # wave2 => real/imag
    real2, imag2 = wave2.to_complex(max_terms=20)
    print("wave2 to_complex => real =", real2, ", imag =", imag2)
    print("   as float => real=%.5f, imag=%.5f" % (real2.to_float(), imag2.to_float()))

    # wave1+wave2 => real/imag
    real_add, imag_add = w_add.to_complex(max_terms=20)
    print("\n(wave1+wave2).to_complex => real=", real_add, ", imag=", imag_add)
    print("   as float => real=%.5f, imag=%.5f" % (real_add.to_float(), imag_add.to_float()))

In [9]:
import mpmath

class WaveMpmath:
    """
    A wave using mpmath mpf for amplitude and phase.
    Wave multiply => amplitude multiply, phase add
    Wave add => complex sum => amplitude + phase
    """

    def __init__(self, amplitude=None, phase=None):
        # Default => amplitude=0, phase=0
        if amplitude is None:
            amplitude = mpmath.mpf('0')
        if phase is None:
            phase = mpmath.mpf('0')
        self.amplitude = mpmath.mpf(amplitude)
        self.phase = mpmath.mpf(phase)

    def multiply(self, other: "WaveMpmath") -> "WaveMpmath":
        new_amp = self.amplitude * other.amplitude
        new_phase = self.phase + other.phase
        return WaveMpmath(new_amp, new_phase)

    def add_wave(self, other: "WaveMpmath") -> "WaveMpmath":
        # convert each wave to complex => sum => convert back
        r1, i1 = self.to_complex()
        r2, i2 = other.to_complex()
        sum_r = r1 + r2
        sum_i = i1 + i2
        # amplitude = sqrt(re^2 + im^2)
        new_amp = mpmath.sqrt(sum_r*sum_r + sum_i*sum_i)
        # phase = atan2(im, re)
        new_phase = mpmath.atan2(sum_i, sum_r)
        return WaveMpmath(new_amp, new_phase)

    def to_complex(self):
        """
        amplitude*(cos(phase) + i*sin(phase))
        """
        r = self.amplitude * mpmath.cos(self.phase)
        i = self.amplitude * mpmath.sin(self.phase)
        return (r, i)

    def __str__(self):
        return f"WaveMpmath(amp={self.amplitude}, phase={self.phase})"

In [10]:
import time
import random

def timing_tests_mpmath(num_trials=20, precision=100):
    """
    Compare performance using mpmath for sin and wave arithmetic.
    :param num_trials: how many random angles/pairs to test
    :param precision: mpmath precision (bits or digits, depending on mp context)
    """

    print(f"\n=== timing_tests_mpmath (num_trials={num_trials}, prec={precision}) ===")

    # Set precision. By default, mp.prec means bits of precision, 
    # or if mp.mp.dps => decimal places. 
    # We'll assume bits; adapt as needed.
    mpmath.mp.prec = precision

    # 1) Sin expansions with mpmath
    start_sin = time.time()
    for _ in range(num_trials):
        angle_float = random.uniform(0, 2.0)
        angle_mpf = mpmath.mpf(angle_float)
        _ = mpmath.sin(angle_mpf)
    end_sin = time.time()
    total_sin = end_sin - start_sin
    print(f"mpmath.sin() for {num_trials} random angles => "
          f"{total_sin:.4f} s, avg {total_sin/num_trials:.4f} s each")

    # 2) Wave multiplication
    start_wave_mult = time.time()
    for _ in range(num_trials):
        ampA = random.uniform(0.1, 3.0)
        phaseA = random.uniform(0, 2.0)
        ampB = random.uniform(0.1, 3.0)
        phaseB = random.uniform(0, 2.0)
        waveA = WaveMpmath(ampA, phaseA)
        waveB = WaveMpmath(ampB, phaseB)
        _ = waveA.multiply(waveB)
    end_wave_mult = time.time()
    total_wave_mult = end_wave_mult - start_wave_mult
    print(f"Wave multiplication (mpmath) for {num_trials} random pairs => "
          f"{total_wave_mult:.4f} s, avg {total_wave_mult/num_trials:.4f} s each")

    # 3) Wave addition => triggers to_complex => sin/cos
    # Actually we call mpmath cos & sin in to_complex
    start_wave_add = time.time()
    for _ in range(num_trials):
        ampA = random.uniform(0.1, 2.0)
        phaseA = random.uniform(0, 2.0)
        ampB = random.uniform(0.1, 2.0)
        phaseB = random.uniform(0, 2.0)
        waveA = WaveMpmath(ampA, phaseA)
        waveB = WaveMpmath(ampB, phaseB)
        _ = waveA.add_wave(waveB)
    end_wave_add = time.time()
    total_wave_add = end_wave_add - start_wave_add
    print(f"Wave addition (mpmath) for {num_trials} random pairs => "
          f"{total_wave_add:.4f} s, avg {total_wave_add/num_trials:.4f} s each")

In [11]:
if __name__ == "__main__":
    # Basic arithmetic tests
    test_bigbase65536_basic()
    test_myfraction_basic()
    test_sqrtd_basic()
    test_wave_basic()

    # Performance tests
    timing_tests(num_trials=20, max_terms=40)
    stress_test(num_trials=10, angle_range=(0, 100.0), max_terms=100)
    timing_tests_mpmath(num_trials=20, precision=100)


=== test_bigbase65536_basic ===
A => <BigBase65536 123456> 
B => <BigBase65536 654321>
A + B => <BigBase65536 777777>
A * B => <BigBase65536 80779853376>
Check => 777777 80779853376

NegA => <BigBase65536 -123456> 
NegB => <BigBase65536 -654321>
NegA + NegB => <BigBase65536 -777777>
NegA * NegB => <BigBase65536 80779853376>
Check => -777777 80779853376

valNegA + valB => <BigBase65536 530865> (Check => 530865 )

=== test_myfraction_basic ===
half_neg => -1/2
three_quarters => 3/4

(-1/2) + (3/4) => 1/4  (~ 0.25 )
(-1/2) * (3/4) => -3/8  (~ -0.375 )

7/3 => 7/3 (~ 2.3333333333333335 )
(7/3) / (3/4) => 28/9 (~ 3.111111111111111 )

=== test_sqrtd_basic ===
x => (-2 + 3*sqrt(5))
x as float => 4.708203932499369

y => (4/5 + -1/3*sqrt(5))
y as float => 0.054644007500070146

x + y => (-6/5 + 8/3*sqrt(5))  (~ 4.762847939999439 )
x * y => (-33/5 + 46/15*sqrt(5))  (~ 0.25727513099935617 )
x / y => (765/19 + 390/19*sqrt(5))  (~ 86.16139532762728 )

=== test_wave_basic ===
wave1 => Wave(amp=3/2, 